In [1]:
import pandas as pd
import os

In [2]:
url = "https://www.sharkattackfile.net/spreadsheets/GSAF5.xls"
original_df = pd.read_excel(url)

In [3]:
# create working copy of original dataframe
df = original_df.copy()

In [4]:
# get first impression of dataset
df.head(3)

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,15 Mar 2024,2024.0,Unprovoked,AUSTRALIA,Queensland,Bargara Beach,Swimming,Brooklyn Sauer,F,13,...,Tiger shark,"Yahoo News, 3/15/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04 Mar 2024,2024.0,Unprovoked,USA,Hawaii,"Old Man's, Waikiki",Surfing,Matthew White,M,NaN,...,Tiger shark 8',"Surfer, 3/6/2024F",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,02 Mar-2024,2024.0,Unprovoked,USA,Hawaii,"Rainbows, Oahu",Swimming,NaN,F,11,...,3' to 4' shark,"Hawaii News Now, 3/4/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.shape

(6969, 23)

In [6]:
# clean column names
df.columns

Index(['Date', 'Year', 'Type', 'Country', 'State', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Unnamed: 11', 'Time', 'Species ',
       'Source', 'pdf', 'href formula', 'href', 'Case Number', 'Case Number.1',
       'original order', 'Unnamed: 21', 'Unnamed: 22'],
      dtype='object')

In [7]:
# remove whitespice behind column name for species
df = df.rename(columns = {"Species ": "Species"})

In [8]:
# choose certain variables that we are of interest for our problem statement
df = df[["Date", "Year", "Country", "Location", "Injury", "Species", "Activity"]]

In [9]:
# drop all rows where all columns are NaN
df = df.dropna(how='all')
df.shape

(6944, 7)

In [10]:
# drop all rows where 'Year' is NaN
df = df.dropna(subset = ['Year'])
df.head()


,Date,Year,Country,Location,Injury,Species,Activity
0,15 Mar 2024,2024.0,AUSTRALIA,Bargara Beach,"Minor injuries to back, abdomen and legs",Tiger shark,Swimming
1,04 Mar 2024,2024.0,USA,"Old Man's, Waikiki","No injury, shark bit surfboard",Tiger shark 8',Surfing
2,02 Mar-2024,2024.0,USA,"Rainbows, Oahu",Lacerations to left foot,3' to 4' shark,Swimming
3,25 Feb-2024,2024.0,AUSTRALIA,"Sandlnd Island, Jurian Bay",Leg bitten,Tiger shark,NaN
4,14 Feb-2024,2024.0,INDIA,"Vaitarna River, Palghar District",Calf of lower left leg injured,"Bull shark, 7'",Fishing


In [11]:
# correct the way the Years are written (for example 2023.0 to 2023) [from string to integer]
df['Year'] = df['Year'].astype(int)
df.head(3)

,Date,Year,Country,Location,Injury,Species,Activity
0,15 Mar 2024,2024,AUSTRALIA,Bargara Beach,"Minor injuries to back, abdomen and legs",Tiger shark,Swimming
1,04 Mar 2024,2024,USA,"Old Man's, Waikiki","No injury, shark bit surfboard",Tiger shark 8',Surfing
2,02 Mar-2024,2024,USA,"Rainbows, Oahu",Lacerations to left foot,3' to 4' shark,Swimming


In [12]:
# only consider last 10 years
df = df[(df['Year'] >= 2015)]

In [13]:
# check how many incidents in each year
df['Year'].value_counts()

2015    143
2017    139
2016    131
2018    124
2019    114
2021    109
2023    108
2020    101
2022     98
2024     10
Name: Year, dtype: int64

In [14]:
# check which countries have high incident number
df.Country.value_counts()[0:5]

USA             508
AUSTRALIA       248
BAHAMAS          43
SOUTH AFRICA     43
BRAZIL           21
Name: Country, dtype: int64

In [16]:
# only consider incidents in USA
df = df.loc[df['Country'] == 'USA']

In [17]:
# check if hotspot exists
df['Location'].value_counts()[0:5]

New Smyrna Beach, Volusia County    48
Cocoa Beach, Brevard  County        13
Ponce Inlet, Volusia County         11
Jacksonville Beach, Duval County     7
Myrtle Beach, Horry County           6
Name: Location, dtype: int64

In [18]:
# explore injury types
df['Injury'].value_counts()[0:10]

Foot bitten                  14
FATAL                         8
Lacerations to left foot      8
Minor injury to foot          7
No injury, kayak bitten       6
Leg bitten                    6
No injury                     5
Lacerations to right foot     5
Minor injuries                4
Left foot bitten              4
Name: Injury, dtype: int64

In [19]:
df['Injury'].nunique()

386

In [20]:
# classify injury types
injury_classification = ["No Injury", "Minor", "Fatal", "Lacerations", "Bitten", "Injured", "Punctures"]
df['Injury'] = df['Injury'].apply(lambda x: "No Injury" if "no injury" in str(x).lower() else x)
df['Injury'] = df['Injury'].apply(lambda x: "No Injury" if "nor injured" in str(x).lower() else x)
df['Injury'] = df['Injury'].apply(lambda x: "Minor" if "minor" in str(x).lower() else x)
df['Injury'] = df['Injury'].apply(lambda x: "Fatal" if "fatal" in str(x).lower() else x)
df['Injury'] = df['Injury'].apply(lambda x: "Lacerations" if "laceration" in str(x).lower() else x)
df['Injury'] = df['Injury'].apply(lambda x: "Bitten" if "bitten" in str(x).lower() else x)
df['Injury'] = df['Injury'].apply(lambda x: "Bitten" if "bite" in str(x).lower() else x)
df['Injury'] = df['Injury'].apply(lambda x: "Injured" if "injured" in str(x).lower() else x)
df['Injury'] = df['Injury'].apply(lambda x: "Injured" if "injury" in str(x).lower() else x)
df['Injury'] = df['Injury'].apply(lambda x: "Injured" if "injuries" in str(x).lower() else x)
df['Injury'] = df['Injury'].apply(lambda x: "Punctures" if "puncture" in str(x).lower() else x)
df['Injury'] = df['Injury'].apply(lambda x: "Punctures" if "puncture" in str(x).lower() else x)
df['Injury'] = df['Injury'].apply(lambda x: "Others" if x not in injury_classification else x)

In [21]:
df['Injury'].value_counts()

Injured        141
Lacerations    117
Bitten          98
Minor           79
Others          42
Punctures       22
Fatal            9
Name: Injury, dtype: int64